Question 1

In [47]:
# Let's read the contents of the provided FASTA file to understand its structure
file_path = 'Ecoli_GCF_003018035.1_ASM301803v1_genomic (1).fna'

# Read the contents of the file
with open(file_path, 'r') as file:
    file_contents = file.readlines()

# Show the first few lines to understand the structure and data
file_contents[:10]

['>NZ_CP027390.1 Escherichia coli strain 2015C-4944 chromosome, complete genome\n',
 'TGCTCTCTTTACCGCTGTTTCACGTCTCCGGTCAGGGAATTATGTGGCGCTGGTTATACGCTGGTGCGCGGATGACGGTG\n',
 'CGTGATAAACAGCCATTGGAGCAAATGCTGGCAGGCTGTACTCATGCTTCACTGGTGCCAACACAACTCTGGCGTTTGCT\n',
 'GGTTAACCGCAGTTCCGTTTCCCTGAAAGCGGTGTTACTTGGCGGCGCGGCTATCCCGGTCGAGTTGACGGAACAGGCGC\n',
 'GCGAGCAGGGGATTCGTTGCTTTTGCGGCTATGGTCTGACCGAGTTTGCCTCCACGGTGTGTGCGAAAGAAGCCGACGGC\n',
 'CTGGCAGACGTTGGTTCGCCGCTGCCGGGTCGGGAAGTGAAAATCGTTAATAATGAAGTGTGGCTGCGGGCTGCCAGTAT\n',
 'GGCAGAAGGTTACTGGCGTAACGGGCAACTGGTTTCACTGGTTAATGACGAAGGCTGGTACGCTACGCGCGATCGCGGTG\n',
 'AGATGCATAATGGCAAGCTGACCATTGTCGGACGTTTAGACAATCTATTCTTCAGTGGCGGAGAGGGTATTCAGCCGGAA\n',
 'GAAGTCGAGCGTGTTATTGCTGCACATCCTGCGGTTTTGCAGGTGTTTATCGTTCCCGTTGCCGACAAGGAGTTTGGTCA\n',
 'TCGGCCGGTAGCGGTGGTGGAGTATGACCAGCAAACCGTTGATCTTGGCGAATGGGTGAAAGATAAGCTGGCTCGTTTTC\n']

In [37]:
# Combine all sequence lines into a single continuous string
nucleotide_sequence = ''.join(line.strip() for line in file_contents[1:])

# Calculate the length of the entire sequence
sequence_length = len(nucleotide_sequence)

# Calculate split points for 80% training and 20% testing
train_length = int(sequence_length * 0.8)
test_length = sequence_length - train_length

# Split the sequence
train_sequence = nucleotide_sequence[:train_length]
test_sequence = nucleotide_sequence[train_length:]

(train_length, test_length, len(train_sequence), len(test_sequence))


(4721244, 1180312, 4721244, 1180312)

In [14]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm  # Import tqdm for the progress bar

# Constants
SEQUENCE_LENGTH = 100
BATCH_SIZE = 256
HIDDEN_DIM = 256
NUM_LAYERS = 2
EPOCHS = 10
LEARNING_RATE = 0.005
VOCAB_SIZE = 4  # A, T, C, G

# Mapping of characters to integers
char_to_int = {'A': 0, 'T': 1, 'C': 2, 'G': 3}
int_to_char = {i: c for c, i in char_to_int.items()}

# Convert sequence to encoded data, filtering out unwanted characters
def encode_sequence(seq):
    return [char_to_int[char] for char in seq if char in char_to_int]

# Preprocessing the dataset
def create_dataset(sequence):
    encoded_seq = encode_sequence(sequence)
    X = []
    Y = []
    for i in range(0, len(encoded_seq) - SEQUENCE_LENGTH):
        X.append(encoded_seq[i:i+SEQUENCE_LENGTH])
        Y.append(encoded_seq[i+SEQUENCE_LENGTH])
    return np.array(X), np.array(Y)

# PyTorch custom dataset
class GenomeDataset(Dataset):
    def __init__(self, sequence):
        self.x, self.y = create_dataset(sequence)
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx], dtype=torch.long).to(device), torch.tensor(self.y[idx], dtype=torch.long).to(device)

# Define the RNN model
class RNNModel(nn.Module):
    def __init__(self, vocab_size, hidden_dim, num_layers):
        super(RNNModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.rnn = nn.RNN(vocab_size, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x):
        # One-hot encode input
        x = nn.functional.one_hot(x, num_classes=VOCAB_SIZE).to(torch.float32)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

# Load data
train_dataset = GenomeDataset(train_sequence)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataset = GenomeDataset(test_sequence)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Check for CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "mps")
# Initialize model
model = RNNModel(VOCAB_SIZE, HIDDEN_DIM, NUM_LAYERS).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training loop
for epoch in range(EPOCHS):
    model.train()
    progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{EPOCHS}')
    for inputs, labels in progress_bar:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        progress_bar.set_postfix(loss=loss.item())

# Evaluation
def evaluate(model, data_loader):
    model.eval()
    total = 0
    correct = 0
    with torch.no_grad():
        for inputs, labels in data_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

# Evaluate the model
accuracy = evaluate(model, test_loader)
print(f'Accuracy on test set: {accuracy}%')


Epoch 3/10:  25%|██▌       | 4693/18442 [12:02<35:15,  6.50it/s, loss=1.4] 


KeyboardInterrupt: 

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "mps")

# Function to generate k-mers
def generate_kmers(sequence, k):
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]
    return kmers

# Create sequences of k-mers
def create_kmer_sequences(kmers):
    X, Y = [], []
    for i in range(len(kmers) - 1):
        X.append(kmers[i])
        Y.append(kmers[i + 1])
    return X, Y

# Define the RNN model
class KmerRNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(KmerRNNModel, self).__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim)
        self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)
        self.out = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = self.out(x[:, -1, :])  # Output of last time step
        return x

# Load and preprocess genomic data
filepath = 'Ecoli_GCF_003018035.1_ASM301803v1_genomic (1).fna' # Replace with your file path
with open(filepath, 'r') as file:
    data = file.readlines()
genomic_sequence = ''.join(line.strip() for line in data[1:])

# Generate k-mers and create training/testing data
k = 4
all_kmers = generate_kmers(genomic_sequence, k)
train_kmers, test_kmers = train_test_split(all_kmers, test_size=0.2, random_state=42)
X_train_kmers, y_train_kmers = create_kmer_sequences(train_kmers)
X_test_kmers, y_test_kmers = create_kmer_sequences(test_kmers)

# Encode k-mers
le = LabelEncoder()
le.fit(all_kmers)
X_train_encoded = torch.tensor(le.transform(X_train_kmers), device=device)
y_train_encoded = torch.tensor(le.transform(y_train_kmers), device=device)
X_test_encoded = torch.tensor(le.transform(X_test_kmers), device=device)
y_test_encoded = torch.tensor(le.transform(y_test_kmers), device=device)

# Model parameters
vocab_size = len(le.classes_) 
embedding_dim = 10
hidden_dim = 50

# Initialize the model, loss criterion, and optimizer
model = KmerRNNModel(vocab_size, embedding_dim, hidden_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training the model
def train_model(model, X_train, y_train, criterion, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        inputs = Variable(X_train.unsqueeze(1))  # Reshape to [batch_size, 1] to match LSTM input
        labels = Variable(y_train)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# Evaluating the model
def evaluate_model(model, X_test, y_test):
    model.eval()
    inputs = Variable(X_test.unsqueeze(1))
    labels = Variable(y_test)
    outputs = model(inputs)
    _, predicted = torch.max(outputs.data, 1)
    correct = (predicted == labels).sum().item()
    accuracy = 100 * correct / len(labels)
    print(f'Test Accuracy: {accuracy:.2f}%')

# Run training and evaluation
train_model(model, X_train_encoded, y_train_encoded, criterion, optimizer)
evaluate_model(model, X_test_encoded, y_test_encoded)


RuntimeError: MPS backend out of memory (MPS allocated: 8.18 GB, other allocations: 771.55 MB, max allowed: 9.07 GB). Tried to allocate 900.51 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).